In [ ]:
propno = 7076
runno = 56
train_slice = slice(0, 100)

detector_id = "SPB_DET_AGIPD1M-1"

result_dir = f"proc/p{propno:06d}/r{runno:04d}"

litpx_counter_id = f"{detector_id}/REDU/LITPX_COUNTER"
spi_hitfinder_id = f"{detector_id}/REDU/SPI_HITFINDER"

adu_threshold = 0.7

modules=[3, 4, 8, 15]
mode="adaptive"
snr=4.0
min_scores=100
fixed_threshold=0
hitrate_window_size=200
miss_fraction=1
miss_fraction_base="hit"

num_proc = 0

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt
import multiprocessing as mp
import psutil

from extra_data import RunDirectory, open_run

from extra_redu.spi import LitPixelCounter, SpiHitfinder
from extra_redu.fileutils import StackedPulseSource, exdf_save

from IPython.display import Latex, Markdown

In [ ]:
display(Markdown("# SPI Hit Finding"))

In [ ]:
try:
    run = RunDirectory(result_dir)
    hitfinder = SpiHitfinder.from_collection(run, spi_hitfinder_id)
    do_hitfinding = False
    display(Markdown("- [x] read results from files\n"
                     "- [ ] analyse images"))
except (FileNotFoundError, KeyError):
    do_hitfinding = True
    display(Markdown("- [ ] read results from files\n"
                     "- [x] analyse images"))

In [ ]:
analysis_steps = []
if do_hitfinding:
    # 1. open run with corrected images
    dc = open_run(propno, runno, data="proc")
    dc = dc.select_trains(train_slice)

    src = StackedPulseSource.from_datacollection(
        dc, detector_id + r"/DET/(?P<key>\d+)CH0:xtdf", "image")

In [ ]:
if do_hitfinding:
    # 2. run lit-pixel counter
    litpx_counter = LitPixelCounter(src, threshold=adu_threshold)

    if not num_proc:
        num_proc = min(psutil.cpu_count(logical=False), 32)

    with mp.Pool(num_proc) as pool:
        chunks = src.split_trains(trains_per_part=1)
        results = pool.imap_unordered(litpx_counter, chunks)
        for _ in results:
            pass

In [ ]:
if do_hitfinding:
    # 3. run hit finding
    hitfinder = SpiHitfinder(
        modules=modules,
        mode=mode,
        snr=snr,
        min_scores=min_scores,
        fixed_threshold=fixed_threshold,
        hitrate_window_size=hitrate_window_size,
        miss_fraction=miss_fraction,
        miss_fraction_base=miss_fraction_base,
    )
    hitfinder.find_hits(litpx_counter)

In [ ]:
if do_hitfinding:
    # write results in file
    os.makedirs(result_dir, exist_ok=True)
    sources = {
        litpx_counter_id: litpx_counter,
        spi_hitfinder_id: hitfinder,
    }
    exdf_save(result_dir, "REDU00", runno, sources, sequence_size=3500)

In [ ]:
display(Markdown("## Hitrate"))
hitfinder.plot_hitrate()
plt.show()

In [ ]:
display(Markdown("## Hitscores histogram"))
hitfinder.plot_hitscore_hist()
plt.show()

In [ ]:
display(Markdown("## Hitscores over trains"))
hitfinder.plot_hitscore()
plt.show()